## IMPORTS

In [ ]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns
sns.set(style = "darkgrid")

In [ ]:
from functions_phase_1 import data_constructor
from functions_phase_1 import load_labels
from functions_phase_1 import dataframe_generator
from functions_phase_1 import index_generator
from functions_phase_1 import train_index_updater

## SETTING UP PARAMETERS AND DIRECTORIES

In [ ]:
SEED = 12
IMG_HEIGHT = 128
IMG_WIDTH = 128
DIM = (IMG_HEIGHT, IMG_WIDTH)
TRAIN_SET= 143778
VALID_SET= 25486
EPOCHS = 40
BS = 32
n = 50000
LR = 0.0001
label_file_train = "C:\project\data\\train_COVIDx_CT-2A.txt"
label_file_valid = "C:\project\data\\val_COVIDx_CT-2A.txt"

## LOADING DATA

In [ ]:
fnames_train , classes_train ,bboxes_train = load_labels(label_file_train)
fnames_valid , classes_valid ,bboxes_valid = load_labels(label_file_valid)
train_index = index_generator(fnames_train , TRAIN_SET)
valid_index = index_generator(fnames_valid , VALID_SET)
train_index_updated = train_index_updater(classes_train,train_index,n )
df = dataframe_generator(train_index_updated , valid_index , classes_train , classes_valid)
df.plot.bar( title = "Image Distribution" );

## Loading Training and Validation Images

In [ ]:
x_train, y_train = data_constructor(fnames_train,classes_train, DIM , index  =train_index_updated ,bboxes = bboxes_train )
x_valid , y_valid= data_constructor(fnames_valid,classes_valid, DIM , index  =valid_index , bboxes = bboxes_valid)

## Pre-Processing

In [ ]:
x_train = tf.keras.applications.densenet.preprocess_input(x_train)
x_valid = tf.keras.applications.densenet.preprocess_input(x_valid)

In [ ]:
for i in range(len(y_train)):
    if y_train[i] ==1:
        y_train[i]=0
    if y_train[i] ==2:
        y_train[i]=1
for i in range(len(y_valid)):
    if y_valid[i] ==1:
        y_valid[i]=0
    if y_valid[i]==2:
        y_valid[i]=1

In [ ]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,zoom_range=0.05,width_shift_range=0.2 ,height_shift_range=0.1,shear_range=0.05)## ENTER YOUR AUG HERE
train_generator = data_generator.flow(x_train, y_train, BS)
spe = x_train.shape[0] // BS

## MODEL DESIGN

In [ ]:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout

In [ ]:
base_model = DenseNet201(include_top=False,weights="imagenet",input_shape=(IMG_HEIGHT,IMG_HEIGHT,3))
model= tf.keras.Sequential()
model.add(base_model) 
model.add(GlobalAveragePooling2D())
model.add(Dense(64,activation=('relu'))) 
model.add(Dropout(.2))
model.add(Dense(1,activation=('sigmoid')))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(LR),
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
modelsdir = "models"
checkpoint = ModelCheckpoint(filepath = modelsdir + "/SGD.{epoch:02d}-{val_accuracy:.4f}.h5" ,
                             monitor= "val_accuracy" , 
                             mode = "max",
                            verbose = 0,
                            save_best_only=True)

In [ ]:
r = model.fit(train_generator , validation_data=(x_valid, y_valid), steps_per_epoch = spe ,epochs=EPOCHS, callbacks = [checkpoint] ,verbose = 2 )

In [ ]:
plt.plot(r.history["loss"], label= "Loss")
plt.plot(r.history["val_loss"], label = "val_Loss")
plt.legend();

In [ ]:
plt.plot(r.history["accuracy"] ,label = "Accuracy")
plt.plot(r.history["val_accuracy"] ,label = "Val_Accuracy")
plt.legend();

In [ ]:
pd.DataFrame(r.history).to_csv("densenet.csv",index=False)

In [ ]:
model.save("Densenet.h5")

## MORE TRAINING

In [ ]:
LR = 0.001
EPOCHS = 30

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(LR),
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
modelsdir = "models"
checkpoint = ModelCheckpoint(filepath = modelsdir + "/(mt)SGD.{epoch:02d}-{val_accuracy:.4f}.h5" ,
                             monitor= "val_accuracy" , 
                             mode = "max",
                            verbose = 0,
                            save_best_only=True)

In [ ]:
s = model.fit(train_generator , validation_data=(x_valid, y_valid), steps_per_epoch = spe ,epochs=EPOCHS, callbacks = [checkpoint] ,verbose = 2 )

In [ ]:
plt.plot(s.history["loss"], label= "Loss")
plt.plot(s.history["val_loss"], label = "val_Loss")
plt.legend();

In [ ]:
plt.plot(s.history["accuracy"] ,label = "Accuracy")
plt.plot(s.history["val_accuracy"] ,label = "Val_Accuracy")
plt.legend();

In [ ]:
pd.DataFrame(s.history).to_csv("densenet-2.csv",index=False)

In [ ]:
model.save("Densenet-2.h5")

### Loading Combined CSV for plots

In [ ]:
history = pd.read_csv("csv/densenet-combined.csv")

In [ ]:
plt.plot(history["loss"], label= "Loss")
plt.plot(history["val_loss"], label = "val_Loss")
plt.legend();

In [ ]:
plt.plot(history["accuracy"] ,label = "Accuracy")
plt.plot(history["val_accuracy"] ,label = "Val_Accuracy")
plt.legend();

In [ ]:
history.plot();